Sacar mi historial de partidas SUMMONER-V4, MATCH-V5
o 
info de los jugadores de una división concreta LEAGUE-V4 division

In [1]:
import requests
import pandas as pd
from creds import api_key

In [2]:
url_summoner = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Masgt7'

Nos centramos en SUMMONER-V4que tiene que ver con cualquier cosa que haga una cuenta en concreto

In [3]:
url_summoner = url_summoner + '?api_key=' + api_key

In [4]:
r = requests.get (url_summoner)
r.status_code

200

In [5]:
player_info = r.json()
print (player_info)

{'id': '7xKp5xDyqk1u0_RKrh8SdaSHYhjpThiq7XXJscT2DQx273I', 'accountId': 'NcGfPSobpg1gNOsZBjrQNCVyeIg7MyijVoglw0WBQioOST0', 'puuid': 'uFIekhnHnZjaIXG_IG5y2Kb_UbBsEv57LksBUe6qMh0h0MPBKg8xoWtsbQkvXcPhlHLNGn8xZwqzGQ', 'name': 'Masgt7', 'profileIconId': 940, 'revisionDate': 1700658317030, 'summonerLevel': 74}


Obtenida la info de mi cuenta, ahora hacemos otra request para obtener identificadores de mis últimas partidas a partir del 'puuid'

In [6]:
puuid = player_info['puuid']

In [7]:
url_match_history = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?type=normal&start=0&count=20&api_key=' + api_key #Limitamos la lista de partidas a 20
r = requests.get(url_match_history)

In [8]:
if r.status_code == 200:
    game_id = r.json()
    print (game_id)
else:
    print ('Error', r.status_code, ':',)  

['EUW1_6686773929', 'EUW1_6686744190', 'EUW1_6684157765', 'EUW1_6684133215', 'EUW1_6682813114', 'EUW1_6682767095', 'EUW1_6682519608', 'EUW1_6678781224', 'EUW1_6678748896', 'EUW1_6678696426', 'EUW1_6678677984', 'EUW1_6678647009', 'EUW1_6677476957', 'EUW1_6677447766', 'EUW1_6673723132', 'EUW1_6673680411', 'EUW1_6673646176', 'EUW1_6673557905', 'EUW1_6673484178', 'EUW1_6673427621']


Vamos a sacar ahora información de cada partida y almacenarla en ficheros .csv por separado

In [9]:
counter = 1
for i in game_id:
    url_game_info = 'https://europe.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + api_key
    game_info = requests.get(url_game_info)
    index_player = game_info.json()['metadata']['participants'].index(puuid)
    game_data = game_info.json()['info']['participants'][index_player]
    game_df = pd.json_normalize(game_data)
    game_df.to_csv ('games\\game' + str(counter) + '.csv', sep = ',', mode = 'x')
    counter += 1

Cargamos una lista con DF a partir de los CSV

In [10]:
list_df = []

for i in range(len(game_id)):
    list_df.append((pd.read_csv('games\\game' + str (i+1) + '.csv')))

In [11]:
for i in range (len(list_df)):
    print (list_df[i].shape)

(1, 264)
(1, 263)
(1, 268)
(1, 269)
(1, 268)
(1, 269)
(1, 269)
(1, 268)
(1, 268)
(1, 267)
(1, 267)
(1, 267)
(1, 267)
(1, 266)
(1, 268)
(1, 268)
(1, 267)
(1, 269)
(1, 270)
(1, 269)


No tienen las mismas columnas, de modo que vamos a quedarnos únicamente con las que tengan en común

In [12]:
common_rows = set.intersection(*(set(df.columns) for df in list_df))